In [1]:
import submitit
from frust.stepper import Stepper
import frust.vis as vis
from pathlib import Path
from tooltoad.chemutils import xyz2mol

# Run single structure

In [2]:
f = Path("../structures/misc/HH.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

vis.MolTo3DGrid(f)

2025-10-29 09:59:23 INFO  frust.stepper: Working dir: .


!!! Warning !!! Distance between atoms 2 and 1 (0.640000 A) is suspicious.
!!! Warning !!! Distance between atoms 2 and 1 (0.640000 A) is suspicious.


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=1,
    mem_gb=5,
):
    from pathlib import Path
    # from nuse import start_monitoring
    # start_monitoring(filter_cgroup=True)
    
    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=True,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df = step.orca(df0, "DFT", {
        "wB97X-D3"  : None,
        "6-31G**"   : None,
        "TightSCF"  : None,
        "SlowConv"  : None,
        "OptTS"     : None,
        #"Freq"      : None,
        "NoSym"     : None,
    })
    
    df = step.orca(df, name="DFT-SP", options={
        "wB97X-D3"      : None,
        "6-31+G**"      : None,
        "TightSCF"      : None,
        "SP"            : None,
        "NoSym"         : None,
    }, xtra_inp_str="""%CPCM\nSMD TRUE\nSMDSOLVENT "chloroform"\nend""")

    df.to_parquet(f"{results_dir}/results_{name}.parquet")

In [5]:
DEBUG           = False
N_CORES         = 64
MEM_GB          = 128
TIMEOUT_MIN     = 1440 
RESULTS_DIR     = "test"

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

name = df0["custom_name"].iloc[0]
executor.update_parameters(slurm_job_name=name)
executor.submit(run_orca_calc, df0, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
print(f"Submitted: {name}")

Submitted: HH
